In [2]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00


In [4]:
pip install langchain[openai]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.2/65.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.1/438.1 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 kB 19.3 MB/s eta 0:00:00
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.3.44
    Uninstalling langsmith-0.3.44:
      Successfully uninstalled langsmith-0.3.44
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.63
    Uninstalling langchain-core-0.3.63:
      Successfully uninstalled langchain-core-0.3.63


In [15]:
pip install langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.6/129.6 kB 3.9 MB/s eta 0:00:00


In [30]:
from pydantic import BaseModel, Field
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser, OutputFixingParser
import json


llm = ChatGroq(
    groq_api_key="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",
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=1.0,
    model_kwargs={"presence_penalty": 1.0}
)


class Product(BaseModel):
    product_description: str = Field(description="The generated product description")


parser = PydanticOutputParser(pydantic_object=Product)
output_fixing_parser = OutputFixingParser.from_llm(parser=parser, llm=llm)


content_template = """
# CONTEXT #
I want to advertise for my company's duty free product catalogue. The SKU Name of the product is {sku_name}, brand of the product is {brand_name} and its category is : {sub_category}.

# OBJECTIVE #
Create new product description for the above product based on the following existing description: {description}. Extract all key points and features from the existing description and write a new description from them. The word count should be similar to the original description.

# STYLE #
The writing style needs focus on the unique aspects of the product - its ingredients, its process, its history. Use descriptive language to evoke sensory experience - taste, smell, touch. Be precise and concise when writing. The content needs to be simple enough for the common man to understand it.
Following words are BLACKLISTED from appearing in the response: 'transformative','tapestry','like',';','-'.
Following words are BLACKLISTED from appearing in the response: 'presents'.
REPLACE the following words with a more descriptive and specific response: 'foster','fostering','all about','is about','think of','like','but also'.

# TONE #
Simple, clear and elegant.

# AUDIENCE #
My company's audience profile is the common man who likes to indulge in the finer things in life every once in a while.

{format_instructions}
"""


content_prompt = PromptTemplate(
    input_variables=["sku_name", "brand_name", "sub_category", "description"],
    template=content_template,
    partial_variables={"format_instructions": parser.get_format_instructions()}
)


chain = content_prompt | llm | output_fixing_parser


def format_as_json_instance(result: Product) -> str:
    """
    Formats the Product model output as a JSON instance conforming to the schema:
    {"properties": {"product_description": {"title": "Product Description", "description": "The generated product description", "type": "string"}}, "required": ["product_description"]}
    """
    json_data = result.model_dump()  # Convert Pydantic model to dict
    return json.dumps(json_data, indent=2)

# Sample input data
input_batch = [{
    "sku_name": "Tiffany Sheer Eau de Toilette",
    "brand_name": "Tiffany",
    "sub_category": "Perfumes",
    "description": (
        "Indulge in Tiffany Sheer Eau de Toilette, a vibrant floral scent that bursts with the sweetness of ylang ylang, "
        "the tartness of black currant, and the subtlety of noble iris. This exquisite perfume presents a sparkling "
        "interpretation of the signature fragrance, evoking a sensory experience that delights the senses."
    )
}]

results = chain.batch(input_batch)


for i, result in enumerate(results):

    print(format_as_json_instance(result))
    print()

{
  "product_description": "Tiffany Sheer Eau de Toilette unfolds as a radiant floral fragrance, masterfully blending the lush sweetness of ylang ylang, the tangy essence of black currant, and the refined depth of noble iris. This sophisticated scent captures the essence of a signature fragrance, inviting you to immerse yourself in a sensory delight that awakens the senses."
}

